In [32]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate

In [33]:
import dotenv
import os
from openai import AzureOpenAI

envpath = os.path.join(os.getcwd(),".env")
dotenv.load_dotenv(envpath)

client = AzureOpenAI(
  azure_endpoint = "https://aidetectives.openai.azure.com/", 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-15-preview"
)

In [34]:
import pprint

In [22]:
complete_prompt = """Given a text consisting of Reporting's potential fraud committed,  generate a chronological sequence of summaries by dividing the text into coherent chunks. Each chunk should represent a distinct event or action taken during the exchange betweenthe bank representatives. 
Maintain all relevant information related to Fraud Type Names of people involved in conversation  Case ID, Customer id, Customer name, Account no, Transactions IDs, Card Number, Transaction ID, Recipient ID, Recipients name, Transaction Amount, Date of Transaction, Type of Purchase, Merchant Name, Payment location, Transactions location and names of the system used in the conversation. 
Ensure that chunks are correlated with each other, following a chronological order, and providing answers or contradictions. You can use the bullet points to describe the events. 
Make sure to Include "Is it a fraud or reported by mistake" Customers behaviours like  Change in spending habits Transactions amount max and min based on your understanding at the end. 
inculde all the actions taken till date on the matters that are came to light.
Only show the most important cause and do not include all relevant details. keep the entire summary concise but use separation for presentation. give brief title for each event. make sure that any time or duration mentioned is included in the summary. also show the event title in bold letters. Try to identify the patterns of fraud committed based on conversation, Examine the information in the emails to determine the duration between the initiation and resolution of the complaint.


email summary can be describe using below format
<p><b>{email title}{timestamp}</b>
    <li>{descr 1}</li>
    <li>{descr 2}</li>
    <li>{descr 3}</li>
    
after all events are described, you should give the below information.

        <b>{People/Parties involved in the fraud}</b>
        <b>{Type of fraud commited}</b> 
In case no inforamation found for the points ignore the field
following is the input email {text} """
    


input_email = """ Certainly! Here's the revised email conversation with dummy names:

Email-1:
Subject: Urgent: Potential Bank Impersonation Fraud Attempt
Dear Bank Customer Service,

I am writing to report a concerning incident that occurred earlier today. I received a phone call from a person claiming to be a representative from your bank, stating that there had been suspicious activity on my account and that I needed to verify my personal and financial information immediately.

The caller was very insistent and used high-pressure tactics, claiming that if I did not provide the requested information, my account would be frozen. They provided a phone number that appeared to be associated with your bank, further adding to the legitimacy of the call.

However, I became suspicious of the situation and did not share any sensitive details. I am concerned that this may be a case of bank impersonation fraud, and I wanted to bring this to your attention right away.

Please advise on the authenticity of this call and the necessary steps I should take to protect my account. I appreciate your prompt assistance in addressing this matter.

Sincerely,
Emily Johnson
emily.johnson@email.com

Email-2:
Subject: Re: Potential Bank Impersonation Fraud Attempt
Dear Emily Johnson,

Thank you for bringing this concerning incident to our attention. We take the security of our customers' accounts very seriously and are committed to addressing this matter promptly.

Based on the details you have provided, it appears that this was indeed an attempt at bank impersonation fraud. Our bank would never contact a customer in this manner, demanding sensitive information over the phone. This is a common tactic used by scammers to trick individuals into disclosing confidential data.

We appreciate that you did not share any personal or financial information with the caller. This was the right decision, as it helps to protect your account from potential misuse.

To further address this issue, we kindly request that you provide us with the phone number used by the caller, as well as any other relevant details you may have. Our security team will investigate this incident and take the necessary steps to prevent similar fraudulent attempts in the future.

In the meantime, we recommend that you closely monitor your account for any suspicious activity and immediately report any unauthorized transactions to us. Additionally, please be vigilant of any unsolicited calls, emails, or messages requesting sensitive information, as these are often signs of attempted fraud.

We are here to support you and ensure the security of your account. Please do not hesitate to contact us if you have any other questions or concerns.

Sincerely,
John Smith
john.smith@bank.com

Email-3:
Subject: Re: Potential Bank Impersonation Fraud Attempt
Dear Bank Customer Service,

Thank you for your prompt response and for taking this matter seriously. I have noted the phone number used by the caller, which is 555-123-4567. I did not provide any personal or financial information to the individual.

I appreciate your guidance on monitoring my account and being vigilant against unsolicited requests for sensitive data. This incident has certainly heightened my awareness of the potential for bank impersonation fraud.

Please let me know if there is any additional information I can provide to assist with your investigation. I look forward to your updates on the steps being taken to address this fraudulent attempt.

Sincerely,
Jane Doe
jane.doe@email.com

Email-4:
Subject: Update on Bank Impersonation Fraud Investigation - Case ID: 123456
Dear Emily Johnson,

Thank you for your cooperation and for providing the phone number used by the fraudulent caller. Our security team has thoroughly investigated this incident and can confirm that it was indeed an attempt at bank impersonation fraud.

We have taken several actions to address this matter:
- The phone number provided has been blocked and added to our fraud prevention system to prevent further misuse.
- We have notified the relevant authorities and are working with them to identify the individuals responsible for this attempted scam.
- We have reviewed our internal processes to identify any potential vulnerabilities that may have been exploited and have implemented additional security measures to strengthen our defenses against such fraudulent activities.

Your vigilance and quick response in this situation have been instrumental in helping us address this issue. We commend you for your proactive approach in protecting your account and for not falling victim to the scammer's tactics.

Moving forward, we will continue to monitor your account closely for any suspicious activity. Please do not hesitate to contact us immediately if you notice anything unusual or if you have any other concerns.

Thank you again for your cooperation and for your trust in our bank. We are committed to safeguarding your financial information and ensuring the security of your account.

Best regards,
Mark Thompson
mark.thompson@bank.com

Email-5:
Subject: Re: Update on Bank Impersonation Fraud Investigation - Case ID: 123456
Dear Mark Thompson,

Thank you for the update. We'll remain vigilant and offer our assistance as needed.

Best regards,
Emily Johnson
emily.johnson@email.com

Email-6:
Subject: Re: Update on Bank Impersonation Fraud Investigation - Case ID: 123456
Dear Emily Johnson,

Understood. We appreciate your collaboration in resolving this matter efficiently.

Best regards,
Mark Thompson
mark.thompson@bank.com

Email-7:
Subject: Re: Update on Bank Impersonation Fraud Investigation - Case ID: 123456
Dear Mark Thompson,

Thank you for the detailed and informative update on the actions taken to address the attempted bank impersonation fraud. Your proactive response and the steps taken to strengthen security measures are reassuring.

As a follow-up, I would like to request some additional guidance on how I can better identify and protect myself from similar fraudulent attempts in the future. Could you please provide me with detailed information on the common tactics used by scammers impersonating banks, as well as any recommended best practices for customers to safeguard their accounts?

I am committed to staying vigilant and want to ensure I have the necessary knowledge to recognize and respond appropriately to any potential fraudulent activities targeting my financial information.

Thank you in advance for your assistance.

Sincerely,
Jane Doe
jane.doe@email.com"""

complete_prompt = complete_prompt + input_email

In [21]:
message_text = [
        {"role": "system", "content": "You are an AI assistant that helps people summrize email threads."},
        {"role": "user", "content": complete_prompt}
    ]
    
completion = client.chat.completions.create(
        model="ai-detectives-gpt316k",
        messages=message_text,
        temperature=0.1,
        max_tokens=3500,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )
generated_text = completion.choices[0].message.content

pprint.pprint(generated_text)


('**Email-1: Urgent: Potential Bank Impersonation Fraud Attempt**\n'
 '- Emily Johnson reports a phone call from a person claiming to be a bank '
 'representative.\n'
 '- Caller demanded personal and financial information, threatening to freeze '
 'the account.\n'
 '- Emily did not share any sensitive details, suspecting bank impersonation '
 'fraud.\n'
 '\n'
 '**Email-2: Re: Potential Bank Impersonation Fraud Attempt**\n'
 '- Bank representative confirms it was an attempt at bank impersonation '
 'fraud.\n'
 '- Bank would never contact customers in this manner.\n'
 "- Emily's decision not to share information was correct.\n"
 '- Bank requests the phone number used by the caller for investigation.\n'
 '\n'
 '**Email-3: Re: Potential Bank Impersonation Fraud Attempt**\n'
 '- Emily provides the phone number used by the fraudulent caller '
 '(555-123-4567).\n'
 '- No personal or financial information was shared.\n'
 '- Emily appreciates guidance on monitoring the account and being vigilan

In [35]:
loader = DirectoryLoader(r"C:\\Users\\vsarjera\\OneDrive - Capgemini\\Desktop\\fraudapp\\ai-detectives\\new dataset\\atm card", glob="**/*.txt", loader_cls=TextLoader)

docs = loader.load()

docs

[Document(page_content="Email 1:\nSubject: Potential ATM Card Fraud Detected - Case ID: #4567\n\nDear Team,\n\nWe have detected potential ATM card fraud involving customer Alice Johnson (Email: alice.johnson@example.com). The customer applied for a loan amount of $500 on 2024-05-01. However, upon further review, we have found discrepancies in the application that raise suspicions of fraudulent activity.\n\nCustomer Details:\n\nCustomer ID: CDE23455\nCustomer Name: Alice Johnson\nAccount Number: ACCT-456-123-789\nEmail: alice.johnson@example.com\nFraud Details:\n\nCase ID: #4567\nTransactions IDs:ATM-TXN-789, ATM-TXN-790\nCard Number: xxxx-xxxx-xxxx-1234\nTransaction Amount: $500\nDate of Transaction: 2024-05-01\nMerchant Name: ABC Store\nPayment Location: New York\nBehavioral Analysis:\nWe have noticed a significant increase in the customer's ATM withdrawals over the past week, indicating a change in withdrawing habits.\n\nPlease advise on the next steps to take in this case.\n\nThank 

In [36]:
llm = AzureOpenAI(azure_deployment="ai-detectives-gpt316k")

In [ ]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)